# Ejemplo práctico: recopilar tweets con la API de Twitter

Twitter se ha convertido en una red social que genera una ingente cantidad de información. No son pocas las investigaciones realizadas sobre esta red social, por lo que puede resultar interesante aprender a trabajar con su interfaz. Para ello vamos a necesitar que Twitter nos conceda acceso y nos proporcione unos *tokens* para poder identificarnos. Estos *tokens* son claves de acceso, igual que las contraseñas. Están ligadas a nuestra cuenta de Twitter, por lo que **JAMÁS** debemos dar esas claves a nadie. Una persona con esas claves puede hacer **cualquier cosa** con nuestra cuenta. Se pueden solicitar los *tokens* siguiendo [estos pasos](https://python-twitter.readthedocs.io/en/latest/getting_started.html). Twitter tarda un tiempo en procesar su solicitud, es normal que tarde al menos una semana o diez días.

Existen muchos paquetes para trabajar con la API de Twitter. Nosotros vamos a usar `python-twitter`. Si no lo tenemos instalado hay que ejecutar la celda siguiente. En caso contrario basta con omitirla.

In [ ]:
pip install python-twitter

Antes de nada vamos a importar los paquetes que se van a usar a lo largo del documento y a definir dos funciones auxiliares. La utilidad de estas funciones se explica en la documentación que hay al principio de éstas.

In [ ]:
import json
import twitter
from openpyxl import Workbook

def generate_oauth(self):
    """
    Genera un nuevo archivo de autoridad a partir de los tokens aportados por el usuario.
    """

    import json

    tokens = {'consumer_key':None, 
              'consumer_secret':None, 
              'access_token_key':None, 
              'access_token_secret':None}

    print('No existe archivo de autoridad o está corrupto. Se creará uno nuevo.')

    for key in tokens.keys():
        token = input('Por favor, escriba su {}.'.format(key))
        tokens[key] = token.strip()

    with open('oauth.json', 'w+') as file:
        file.write(json.dumps(tokens))

    # Esto comprueba que todo ha ido bien
    with open('oauth.json', 'r') as file:
        oauth = json.load(file)
    if oauth == tokens:
        print('El archivo se ha creado correctamente.')
        
def connected(reference = 'http://www.google.es'):
    """
    Determina si hay conexión a internet o no intentando conectar a http://www.google.es.
    
    Devuelve True si la hay y False en caso contrario.
    """
    import urllib
    
    try:
        urllib.request.urlopen(reference, timeout=1)
        return True
    except urllib.request.URLError:
        return False

def filter_statuses(raw_statuses, fields):
    """
    Filtra los estados de tweets en función de los campos "fields", que representan los campos de interés.
    
    raw_statuses: list, lista que contiene cada estado de tweets en forma de diccionario.
    
    output: diccionario x : y donde x es la id (str) de cada tweet e y es un diccionario con los campos de interés.
    """
    
    from datetime import datetime as t
    
    statuses = {status['id_str']:{field:status[field] for field in fields if field != 'id_str'} for status in raw_statuses}
    
    for status in statuses:
        statuses[status]['user'] = '{}'.format(statuses[status]['user'].screen_name)

        # Adds current time as one of the fields
        statuses[status]['collected_at'] = str(t.now())
    
    return statuses

Tras tener todo definido e importado lo primero es comprobar si ya existen *tokens* para identificarnos. Si ya existe un fichero `json` con estos datos, entonces la siguiente celda se ejecuta sin problema. Si no existe, nos pedirá token por token y creará este fichero. En el futuro no será necesario volver a introducirlos, ya que el código cargará el fichero `json` que ha guardado.

En la siguiente celda se usa `try`:
Se puede leer sobre el *statement* `try` en [el siguiente enlace](https://docs.python.org/3/reference/compound_stmts.html#try).

In [ ]:
try:
    with open('oauth.json', 'r') as file:
        oauth = json.load(file)

except:
    generate_oauth()
    with open('oauth.json', 'r') as file:
        oauth = json.load(file)

Ahora se inicializa la clase `Api` con los tokens y dos parámetros más:
* `tweet_mode='extended'` nos sirve para obtener toda la información de cada tweet.
* `sleep_on_rate_limit=True` es importante, puesto que nos ayuda a no sobrepasar los límites que Twitter impone para su API. Si estos límites se sobrepasan es posible que Twitter bloquee nuestra cuenta.

Se puede leer más sobre los recursos que proporciona el paquete `python-twitter` en [el siguiente enlace](https://python-twitter.readthedocs.io/en/latest/searching.html) y en [la documentación de sus módulos](https://python-twitter.readthedocs.io/en/latest/twitter.html).

In [ ]:
api = twitter.Api(consumer_key=oauth['consumer_key'],
                  consumer_secret=oauth['consumer_secret'],
                  access_token_key=oauth['access_token_key'],
                  access_token_secret=oauth['access_token_secret'],
                  tweet_mode='extended',
                  sleep_on_rate_limit=True)

Ahora vamos a especificar la cuenta de Twitter (usando su screen_name) y el hashtag que queremos usar para la búsqueda. Ambos en forma de string y sin usar los símbolos que los caracterizan (`@` y `#`). Posteriormente definimos una lista con los campos de los tweets que nos interesan. Podemos consultar qué otros campos hay cuando buscamos tweets en [el ejemplo que hay en el siguiente enlace](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets). Si usamos otro tipo de búsqueda el resultado sería distinto, por lo que los campos cambiarían.

In [ ]:
screen_name = '24h_tve'
hashtag = 'COVID19'
fields = ['created_at', 'favorite_count', 'id_str', 'retweet_count', 'full_text', 'user']

En la siguiente celda se realiza la solicitud a Twitter para que nos envíe la información sobre la búsqueda que queremos hacer. En `query` se ha elaborado la solicitud de búsqueda utilizando los distintos parámetros que permite la API. Se ha utilizado una búsqueda de tipo `raw_query`. La solicitud no es más que una url, por lo que los parámetros vienen separados por `&`. De esta forma se aprecia que se utilizan los siguientes parámetros:
* Con `q=from%3Ascreen_name%20%23hashtag-filter%3Aretweets` se indica la búsqueda que se quiere realizar. Está codificado como una url, de ahí que no sea muy legible. Téngase en cuenta que en el código se ha utilizado `str.format()`, de ahí que en lugar de ´screen_name´ y ´hashtag´, haya ´{}´. Es necesario que esté codificado como una url para que funcione correctamente, pero es conveniente escribirla primero sin codificar para comprobar que nuestra consulta es correcta. [Aquí](https://www.urldecoder.org/) se puede encontrar un recurso online para codificar a url de forma gratuita. Si decodificamos esta consulta, obtendremos `from:screen_name #hashtag-filter:retweets`. De la cuenta `screen_name` estamos pidiendo tweets que contengan el `hashtag` y con el operador `-filter` estamos pidiendo que excluya los `retweets`.
* Con `result_type=popular` le estamos especificando que únicamente nos interesan los tweets más importantes en cuanto a favs y retweets. Esto reduce el número de tweets que conseguimos, pero éstos serán más relevantes.
* Con `lang=es` indicamos que únicamente nos interesan tweets en español.
* Con `count=100` especificamos que queremos 100 tweets. Cada consulta única para la búsqueda `raw_query` de la API tiene un límite de 100 tweets. Otras clases de búsqueda tienen otros límites. Por ejemplo, si usamos el *endpoint* `statuses/user_timeline`, el límite será 200.

Antes de realizar la consulta se comprueba si hay conexión a internet. Si no la hay se indica en el documento y no se realiza la consulta. En la último lugar se transforma la respuesta de la API para trabajar con más comodidad.

* Se puede leer más sobre los límites, los parámetros y los operadores para la búsqueda `raw_query` [aquí](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).
* Se puede leer sobre los límites, los parámetros y los operadores para el *endpoint* `statuses/user_timeline` [aquí](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline).
* Se puede leer más sobre otros *endpoints* [aquí](https://developer.twitter.com/en/docs/api-reference-index).

In [ ]:
if connected():
    query = "q=from%3A{}%20%23{}-filter%3Aretweets&result_type=popular&lang=es&count=100".format(screen_name, hashtag)

    raw_statuses = [status.__dict__ for status in api.GetSearch(raw_query=query)]
    
else:
    print('No se detecta conexión a internet.')

La respuesta de la API contiene muchos campos y puede que no todos nos interesen. En la siguiente celda utilizamos una de las funciones auxiliares que se definieron al principio para filtrar los campos para que únicamente salgan los que hemos especificado en `fields`.

In [ ]:
if len(raw_statuses) > 0:
    # Se filtran los resultados para obtener únicamente los campos de interés
    statuses = filter_statuses(raw_statuses, fields)

else:
    statuses = {}

Twitter impone unos límites que pueden cambiar en el tiempo. Los límites consisten en un número de solicitudes que no podemos superar. Estos límites se reinician cada 15 minutos. Para algunos *endpoints* puede haber otra clase de límites con otros marcos temporales y Twitter es libre de cambiarlos, por lo que antes de usar un código hay que comprobar que se han previsto los límites vigentes. Los paquetes suelen gestionar estos límites para evitar problemas, pero es conveniente ir controlandolos. La siguiente celda comprueba estado de éstos y nos indica cuántas solicitudes restantes tenemos.

In [ ]:
search_limit = api.rate_limit.resources['search']['/search/tweets']['limit']
search_remaining = api.rate_limit.resources['search']['/search/tweets']['remaining']
app_limit = api.rate_limit.resources['application']['/application/rate_limit_status']['limit']
app_remaining = api.rate_limit.resources['application']['/application/rate_limit_status']['remaining']

print('Solicitudes restantes (search): {} / {}'.format(search_remaining, search_limit))
print('Solicitudes restantes (rate limit status): {} / {}'.format(app_remaining, app_limit))

Ahora vamos a comprobar si hemos obtenido algún tweet. La siguiente celda nos muestra el número de tweets que se han recopilado.

In [ ]:
print(len(statuses))

Finalmente vamos a guardar nuestros resultados en un fichero excel que contenga el nombre de la cuenta.

In [ ]:
wb = Workbook()
ws = wb.active

headers = list(statuses[list(statuses.keys())[0]].keys())
headers.append('id')
ws.append(headers)

for tw_id, status in statuses.items():
    data = list(status.values())
    data.append(tw_id)
    
    ws.append(data)

wb.save('{} tweets.xlsx'.format(screen_name))